In [ ]:
import subprocess
import os
import re

# Retrieve database details from environment variables
DB_HOST = os.getenv('DB_HOST', 'default_host')
DB_PORT = os.getenv('DB_PORT', '5432')
DB_NAME = os.getenv('DB_NAME', 'default_db_name')
DB_USER = os.getenv('DB_USER', 'default_user')
DB_PASSWORD = os.getenv('DB_PASSWORD', 'default_password')

# Print the values for debugging
print("DB_HOST:", DB_HOST)
print("DB_PORT:", DB_PORT)
print("DB_NAME:", DB_NAME)
print("DB_USER:", DB_USER)
# Be cautious with printing the password. Uncomment the next line only if absolutely necessary for debugging.
# print("DB_PASSWORD:", DB_PASSWORD)

# Path to the file containing SQL queries, adjusted to read from the /app/outputs directory
query_file_path = "/app/outputs/query_log.txt"
# Initialize a dictionary to store line counts for each table
table_line_counts = {}
# Function to extract the table name from the SQL query
# Function to extract table name from a line
def extract_table_name_from_line(line):
    if line.startswith("Table:"):
        return line.split("Table:")[1].strip()
    return None

# Read the file and execute queries
with open(query_file_path, "r") as file:
    lines = file.readlines()

# Initialize variables
current_table_name = None
query = ""

# Iterate through each line in the file
for line in lines:
    line = line.strip()
    if not line:
        continue  # Skip empty lines
    
    # Extract table name if the line starts with "Table:"
    if line.startswith("Table:"):
        # Process the previous query if it exists
        if query and current_table_name:
            output_file = f"/app/outputs/output_{current_table_name}.csv"
            command = [
                "psql",
                "-h", DB_HOST,
                "-p", str(DB_PORT),
                "-d", DB_NAME,
                "-U", DB_USER,
                "-c", f"\\copy ({query}) TO '{output_file}' WITH CSV HEADER;"
            ]
            try:
                print("Executing command:", " ".join(command))
                subprocess.run(command, env={"PGPASSWORD": DB_PASSWORD}, check=True)
                # Count the number of lines in the output file
                count_command = f"wc -l < {output_file}"
                line_count = subprocess.check_output(count_command, shell=True).decode().strip()
                # Subtract one to account for the header
                table_line_counts[current_table_name] = max(0, int(line_count) - 1)
            except Exception as e:
                print(f"Error executing query for {current_table_name}: {e}")

        # Reset for the next table
        current_table_name = extract_table_name_from_line(line)
        query = ""
    else:
        # Accumulate lines for SQL query
        query += line + " "
# Create a final output file listing each table with its line count
with open("/app/outputs/table_line_counts.txt", "w") as f:
    for table_name, count in table_line_counts.items():
        f.write(f"{table_name}: {count}\n")
# Handle the last query in the file
if query and current_table_name:
    output_file = f"/app/outputs/output_{current_table_name}.csv"
    command = [
        "psql",
        "-h", DB_HOST,
        "-p", str(DB_PORT),
        "-d", DB_NAME,
        "-U", DB_USER,
        "-c", f"\\copy ({query}) TO '{output_file}' WITH CSV HEADER;"
    ]
    try:
        print("Executing command:", " ".join(command))
        subprocess.run(command, env={"PGPASSWORD": DB_PASSWORD}, check=True)
        count_command = f"wc -l < {output_file}"
        line_count = subprocess.check_output(count_command, shell=True).decode().strip()
        # Subtract one to account for the header
        table_line_counts[current_table_name] = max(0, int(line_count) - 1)
    except Exception as e:
        print(f"Error executing query for {current_table_name}: {e}")
# Create a final output file listing each table with its line count
with open("/app/outputs/table_line_counts.txt", "w") as f:
    for table_name, count in table_line_counts.items():
        f.write(f"{table_name}: {count}\n")

DB_HOST: default_host
DB_PORT: 5432
DB_NAME: default_db_name
DB_USER: default_user
DB_PASSWORD: default_password


FileNotFoundError: [Errno 2] No such file or directory: '/backups/query_log.txt'